In [5]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, DatePicker, TimePicker, SelectMultiple, Layout
import numpy as np
import pandas as pd
import datetime
from utils.FileBuffer import FileBuffer
%matplotlib inline


In [6]:
buffer = FileBuffer("data/data_p6_e20.csv", batch_size= 10000)
buffer.next()
columns = buffer.get_columns()
strategies = buffer.get_strategies()

In [7]:

def generate_plot(start_date, start_hour, end_date, end_hour, tags, strategy):
    start = datetime.datetime.combine(start_date, start_hour)
    end = datetime.datetime.combine(end_date, end_hour)
    tags = np.array(tags).flatten()
    strategy = np.array(strategy).flatten()
    data_filter = buffer.get_by_dates(start, end, strategy[0])
    fig, ax = plt.subplots(figsize=(20,10))
    x_values = data_filter["time"]
    for i in range(len(tags)):
        y_values = data_filter[tags[i]]
        line, = ax.plot(x_values, y_values, label=tags[i], lw = 3)
    
    plt.xticks(rotation=45, fontsize = 15)
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%d-%b\n%H:%M:%S'))
    plt.legend()
    plt.show()
    print(len(data_filter))

start_date = DatePicker(value=buffer.init_date.date(), description='Inicio: ')
start_hour = TimePicker(value=buffer.init_date.time(), description='Hora:')
end_date = DatePicker(value=buffer.end_date.date(), description='Fin: ')
end_hour = TimePicker(value=buffer.end_date.time(), description='Hora:')
strategies_sele = SelectMultiple( options=strategies, value=[strategies[0]],  description='Strategies:')
tags = SelectMultiple( options=columns, value=[columns[0]],  description='Tags:')
tags.layout = Layout(height = '400px', width = '500px')
params = {'start_date': start_date, 
          'start_hour': start_hour,
          'end_date': end_date, 
          'end_hour': end_hour, 
          'tags': tags,
          'strategy': strategies_sele}
interactive = widgets.interactive_output(generate_plot, params)
widgets.VBox([
    widgets.HBox([start_date, start_hour]),
    widgets.HBox([end_date, end_hour]),
    widgets.HBox([interactive,tags]),
    widgets.HBox([strategies_sele])
])
